In [1]:
# Useful starting lines
%matplotlib inline
from scipy.sparse import *
import numpy as np
import pickle
import random
import sys
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Data input and output paths
POS_TRAIN_PATH = 'data/twitter-datasets/train_pos_full.txt' 
NEG_TRAIN_PATH = 'data/twitter-datasets/train_neg_full.txt' 
DATA_TEST_PATH = 'data/twitter-datasets/test_data.txt'
OUTPUT_PATH = 'predictions_out.csv'

# For debugging purpose
np.set_printoptions(threshold= 50)
np.random.seed(10)

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
''' DON'T RUN, YOU FOOL
!sh ./scripts/build_vocab.sh
!sh ./scripts/cut_vocab.sh
!python3 src/pickle_vocab.py
!python3 src/cooc.py
'''

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

## Analyse Data


### Null values

In [3]:
print("loading cooccurrence matrix")
#with open('cooc.pkl', 'rb') as f:
with open('saved_gen_files/cooc.pkl', 'rb') as f:
    cooc = pickle.load(f)
print("{} nonzero entries".format(cooc.nnz))

nmax = 100
print("using nmax =", nmax, ", cooc.max() =", cooc.max())

print("initializing embeddings")
embedding_dim = 20
xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
ys = np.random.normal(size=(cooc.shape[1], embedding_dim))

loading cooccurrence matrix
46597044 nonzero entries
using nmax = 100 , cooc.max() = 2599902
initializing embeddings


In [4]:
eta = 0.001
alpha = 3 / 4

epochs = 10

for epoch in range(epochs):
    print("epoch {}".format(epoch))
    for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
        logn = np.log(n)
        fn = min(1.0, (n / nmax) ** alpha)
        x, y = xs[ix, :], ys[jy, :]
        scale = 2 * eta * fn * (logn - np.dot(x, y))
        xs[ix, :] += scale * y
        ys[jy, :] += scale * x
        
np.save('embeddings', xs)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


### Plot Correlation Matrix

In [10]:
embeddings = np.load('embeddings.npy')

print(embeddings)

[[ 4.97596764e-03  3.31558768e-02  1.82222448e-02 ... -6.31126711e-03
  -2.55873538e-02 -2.94767105e-02]
 [-3.00874427e-04  4.66654240e-03 -4.44352661e-03 ...  1.64744560e-02
  -9.34795595e-03  2.00125641e-03]
 [ 1.29250916e-02  3.59479887e-02 -9.74959949e-03 ...  7.05065309e-03
  -1.05950219e-02 -2.66412996e-02]
 ...
 [-2.99275920e-01  1.37295823e+00  1.55530044e+00 ...  4.75312677e-01
   9.34482620e-01  6.53499463e-01]
 [ 1.63393622e+00  1.23704208e+00 -9.68441471e-01 ...  6.80041984e-01
  -1.69755313e+00 -1.89787423e-01]
 [-1.53815961e-01 -1.08982402e-01 -4.59099157e-01 ...  6.72224692e-01
   2.03290847e-01  7.98518058e-01]]


### Outliers 